# Intro: 0_Load_Data

The primary objective of this notebook is to facilitate the loading of data from the "raw data" directory. The process involves importing, cleaning (where necessary), and saving the refined datasets into the "clean data" folder to enable subsequent analysis.

Throughout this notebook, we will:

-Import experimental data and synthetically generated data.

-Perform necessary data cleaning operations to ensure data quality and consistency.

-Store the cleaned data.

# Libraries

In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import f1_score, mean_squared_error, r2_score, make_scorer, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm, neighbors
from sklearn.svm import SVR
from sklearn.model_selection import ShuffleSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern, RationalQuadratic, ConstantKernel

# Load batchs

In [60]:
def load_data_from_excel(file_path):
    """
    Loads data from a specified Excel file.

    Parameters:
    - file_path: String representing the path to the Excel file to be loaded.

    Returns:
    - DataFrame containing the data loaded from the Excel file.
    """
    
    # Load the data from the Excel file
    data = pd.read_excel(file_path)
    
    return data


In [61]:
# Example usage:
# Update these paths with the actual paths to your data files.
address_batch0 = "data\\raw\\batch_0_raw.xlsx"
address_batch1 = "data\\raw\\batch_1_raw.xlsx"
address_batch2 = "data\\raw\\batch_2_raw.xlsx"

# Load the data for each batch using the function
raw_data_batch0 = load_data_from_excel(address_batch0)  # Load data for batch 0
raw_data_batch1 = load_data_from_excel(address_batch1)  # Load data for batch 1
raw_data_batch2 = load_data_from_excel(address_batch2)  # Load data for batch 2

# These DataFrames (raw_data_batch0, raw_data_batch1, raw_data_batch2) are now ready for further processing.


In [62]:
raw_data_batch0.head()

,Experiment ID,Sample ID,Initial_C_Measured,Initial_N_Measured,Initial_Li_Measured,Temperature (C),Final_Li_Measured,%Yield
0,CS-NRCan-014,A1,0.5,4.5,1.0,66,0.720406,0.279594
1,CS-NRCan-014,A2,1.0,4.5,1.0,66,0.660309,0.339691
2,CS-NRCan-014,A3,1.5,4.5,1.0,66,0.735669,0.264331
3,CS-NRCan-014,A5,1.0,6.0,1.0,66,0.621322,0.378678
4,CS-NRCan-014,A6,1.5,6.0,1.0,66,0.655983,0.344017


In [63]:
raw_data_batch1.head()

,Experiment ID,Sample ID,Initial_C_Measured,Initial_N_Measured,Initial_Li_Measured,Final_Li_Measured,%Yield,Initial_C_AI,Initial_L_AI
0,CS-NRCan-CO2-003,B6,2.488,6.00,2.735,0.73,0.70,2.50,3.00
1,CS-NRCan-CO2-003,B2,1.980,6.00,2.498,0.71,0.69,2.00,2.75
2,CS-NRCan-CO2-003,C5,2.536,5.25,2.498,0.76,0.64,2.50,2.75
3,CS-NRCan-CO2-003,B5,2.170,5.75,2.020,0.78,0.57,2.25,2.25
4,CS-NRCan-CO2-003,C4,2.550,5.75,3.450,0.69,0.77,2.50,3.75


In [64]:
raw_data_batch2.head()

,Experiment ID,Sample ID,Initial_C_Measured,Initial_N_Measured,Initial_Li_Measured,Final_Li_Measured,%Yield,Initial_C_AI,Initial_L_AI
0,CS-NRCan-CO2-004,A1,0.310000,5.33,0.83,0.65,0.22,0.50,0.83
1,CS-NRCan-CO2-005,B1,0.924253,2.17,2.00,0.90,0.55,1.00,2.00
2,CS-NRCan-CO2-006,A2,0.749008,3.50,0.50,0.50,0.00,1.00,0.50
3,CS-NRCan-CO2-007,A3,0.996521,3.83,2.33,0.85,0.64,1.17,2.33
4,CS-NRCan-CO2-008,B5,1.368558,3.00,3.00,1.08,0.64,1.50,3.00


# Format correction for data batches


The objective of this section is to standardize the format of all input data frames to ensure consistency, facilitating the comparison of data across different batches. This involves a series of steps to unify the data representation, particularly focusing on the treatment of carbon and nitrogen compounds which are handled differently across batches.

Key differences observed include:

In batch0, the concentrations of carbon-containing and nitrogen-containing compounds are recorded separately.
Conversely, batch1 aggregates these values, not distinguishing between carbon and nitrogen concentrations.
To address these disparities, we will reformat all datasets to reflect total amounts of carbon, nitrogen, and additionally, lithium. This step ensures that each data frame adheres to a uniform structure, making comparative analysis straightforward.


Expected columns in the standardized data frames include:

experiment_id: Unique identifier for each experiment.
init_C: Initial carbon concentration.
init_N: Initial nitrogen concentration.
init_Li: Initial lithium concentration.
T: Temperature of the experiment.
fini_Li: Final lithium concentration post-experiment.
yield: Output yield of the process.
This standardized approach ensures data from various batches can be compared effectively, laying the groundwork for robust analysis and insights.



In [65]:
def clean_batch_data(raw_data):
    """
    Cleans a given batch of data by dropping NA values, calculating initial concentrations,
    and generating new columns for experiment ID and yield ratios.

    Parameters:
    - raw_data: DataFrame containing the raw data for a batch.

    Returns:
    - DataFrame with cleaned and formatted data for the batch.
    """

    # Drop NA values from the raw data
    cleaned_data = raw_data.dropna()

    # Initialize a new DataFrame to hold the cleaned batch data
    new_batch = pd.DataFrame()

    # Calculate the initial carbon concentration
    new_batch['init_C'] = cleaned_data['Initial_C_Measured']  # Adjust column name as per your dataset

    # Calculate the total nitrogen concentration
    new_batch['init_N'] = cleaned_data['Initial_N_Measured']  # Adjust column names as per your dataset

    # Set initial Lithium concentration
    new_batch['init_Li'] = cleaned_data['Initial_Li_Measured']  # Adjust column name as per your dataset

    # Set temperature
    if 'Temperature (C)  ' in cleaned_data.columns:
        new_batch['T'] = cleaned_data['Temperature (C)  ']  # Adjust column name as per your dataset
    else: 
        new_batch['T'] = 66  # Set a default temperature if not available as all experiments were conducted at the same temperature for batches not having temperature data

    # Set final Lithium concentration
    new_batch['fini_Li'] = cleaned_data['Final_Li_Measured']  # Adjust column name as per your dataset

    # Calculate yield ratio
    new_batch['yield'] = (new_batch['init_Li'] - new_batch['fini_Li']) / new_batch['init_Li']


    # If applicable, adjust or remove the Telescope ID generation as per your dataset specifics
    new_batch['Sample_id'] = cleaned_data['Experiment ID'] + '_' + cleaned_data['Sample ID']  # Adjust column names as per your dataset

    # Sort the DataFrame columns
    new_batch = new_batch[['Sample_id', 'init_C', 'init_N', 'init_Li', 'T', 'fini_Li', 'yield']]

    return new_batch

# Example usage:
# cleaned_batch0 = clean_batch_data(raw_data_batch0, "0")
# Note: Replace 'raw_data_batch0' with your actual raw data DataFrame variable for each batch.


## Batch 0: 

In [66]:
new_batch0 = clean_batch_data(raw_data_batch0)
new_batch0.head()

,Sample_id,init_C,init_N,init_Li,T,fini_Li,yield
0,CS-NRCan-014_A1,0.5,4.5,1.0,66,0.720406,0.279594
1,CS-NRCan-014_A2,1.0,4.5,1.0,66,0.660309,0.339691
2,CS-NRCan-014_A3,1.5,4.5,1.0,66,0.735669,0.264331
3,CS-NRCan-014_A5,1.0,6.0,1.0,66,0.621322,0.378678
4,CS-NRCan-014_A6,1.5,6.0,1.0,66,0.655983,0.344017


## Batch 1:

In [67]:
new_batch1 = clean_batch_data(raw_data_batch1)
new_batch1.head()

,Sample_id,init_C,init_N,init_Li,T,fini_Li,yield
0,CS-NRCan-CO2-003_B6,2.488,6.00,2.735,66,0.73,0.733090
1,CS-NRCan-CO2-003_B2,1.980,6.00,2.498,66,0.71,0.715773
2,CS-NRCan-CO2-003_C5,2.536,5.25,2.498,66,0.76,0.695757
3,CS-NRCan-CO2-003_B5,2.170,5.75,2.020,66,0.78,0.613861
4,CS-NRCan-CO2-003_C4,2.550,5.75,3.450,66,0.69,0.800000


# Batch 2

In [68]:
new_batch2 = clean_batch_data(raw_data_batch2)
new_batch2.head()

,Sample_id,init_C,init_N,init_Li,T,fini_Li,yield
0,CS-NRCan-CO2-004_A1,0.310000,5.33,0.83,66,0.65,0.216867
1,CS-NRCan-CO2-005_B1,0.924253,2.17,2.00,66,0.90,0.550000
2,CS-NRCan-CO2-006_A2,0.749008,3.50,0.50,66,0.50,0.000000
3,CS-NRCan-CO2-007_A3,0.996521,3.83,2.33,66,0.85,0.635193
4,CS-NRCan-CO2-008_B5,1.368558,3.00,3.00,66,1.08,0.640000


# Save cleaned data

In [59]:
# Save the dataframes to excel files
new_batch0.to_excel('data\\clean\\batch0.xlsx', index=False)
new_batch1.to_excel('data\\clean\\batch1.xlsx', index=False)
new_batch2.to_excel('data\\clean\\batch2.xlsx', index=False)